# Intermediate Tutorial 2: REST API Integration Patterns

**Level:** Intermediate  
**Time:** 20-25 minutes  
**Prerequisites:** Basic tutorials

## Overview

Professional REST API integration:
- Authentication best practices
- Token refresh strategies
- Rate limiting and retry logic
- Error handling patterns
- Batch operations
- API client wrapper

## Setup

In [ ]:
import requests
import time
from datetime import datetime, timedelta
from typing import Optional, Dict, Any

BASE_URL = "http://localhost:8000/api/v1"

## Step 1: Professional API Client

In [ ]:
class NeuroGraphClient:
    """Production-ready API client with auto-refresh and retry logic."""
    
    def __init__(self, base_url: str, username: str, password: str):
        self.base_url = base_url
        self.username = username
        self.password = password
        self.access_token: Optional[str] = None
        self.refresh_token: Optional[str] = None
        self.token_expires_at: Optional[datetime] = None
        
    def login(self):
        """Initial authentication."""
        response = requests.post(
            f"{self.base_url}/auth/login",
            json={"username": self.username, "password": self.password}
        )
        response.raise_for_status()
        
        data = response.json()
        self.access_token = data["access_token"]
        self.refresh_token = data["refresh_token"]
        self.token_expires_at = datetime.now() + timedelta(seconds=data["expires_in"])
        
        print(f"✓ Logged in as {data['user']['username']}")
    
    def refresh_access_token(self):
        """Refresh expired access token."""
        response = requests.post(
            f"{self.base_url}/auth/refresh",
            json={"refresh_token": self.refresh_token}
        )
        response.raise_for_status()
        
        data = response.json()
        self.access_token = data["access_token"]
        self.refresh_token = data["refresh_token"]
        self.token_expires_at = datetime.now() + timedelta(seconds=data["expires_in"])
        
        print("✓ Token refreshed")
    
    def ensure_valid_token(self):
        """Ensure access token is valid, refresh if needed."""
        if not self.access_token:
            self.login()
        elif datetime.now() >= self.token_expires_at - timedelta(seconds=30):
            self.refresh_access_token()
    
    def request(self, method: str, endpoint: str, **kwargs) -> requests.Response:
        """Make authenticated request with auto-retry."""
        self.ensure_valid_token()
        
        headers = kwargs.get("headers", {})
        headers["Authorization"] = f"Bearer {self.access_token}"
        kwargs["headers"] = headers
        
        url = f"{self.base_url}{endpoint}"
        
        # Retry logic
        max_retries = 3
        for attempt in range(max_retries):
            try:
                response = requests.request(method, url, **kwargs)
                
                # Retry on 401 (token might have expired mid-request)
                if response.status_code == 401 and attempt < max_retries - 1:
                    self.refresh_access_token()
                    headers["Authorization"] = f"Bearer {self.access_token}"
                    continue
                
                response.raise_for_status()
                return response
                
            except requests.exceptions.RequestException as e:
                if attempt == max_retries - 1:
                    raise
                time.sleep(2 ** attempt)  # Exponential backoff
    
    # Convenience methods
    def get(self, endpoint: str, **kwargs):
        return self.request("GET", endpoint, **kwargs)
    
    def post(self, endpoint: str, **kwargs):
        return self.request("POST", endpoint, **kwargs)
    
    def put(self, endpoint: str, **kwargs):
        return self.request("PUT", endpoint, **kwargs)
    
    def delete(self, endpoint: str, **kwargs):
        return self.request("DELETE", endpoint, **kwargs)

# Create client
client = NeuroGraphClient(BASE_URL, "admin", "admin")
client.login()

## Step 2: Using the Client

In [ ]:
# Create token
response = client.post("/tokens", json={
    "position": [1.0]*8,
    "radius": 1.0,
    "weight": 1.0
})
token = response.json()
print(f"✓ Created token {token['token_id']}")

# Get token
response = client.get(f"/tokens/{token['token_id']}")
print(f"✓ Retrieved: {response.json()}")

# Delete token
client.delete(f"/tokens/{token['token_id']}")
print("✓ Deleted")

## Step 3: Rate Limiting Handler

In [ ]:
from collections import deque
from time import time

class RateLimiter:
    """Token bucket rate limiter."""
    
    def __init__(self, max_requests: int, window_seconds: int):
        self.max_requests = max_requests
        self.window = window_seconds
        self.requests = deque()
    
    def acquire(self):
        """Wait if rate limit exceeded."""
        now = time()
        
        # Remove old requests
        while self.requests and self.requests[0] < now - self.window:
            self.requests.popleft()
        
        # Check limit
        if len(self.requests) >= self.max_requests:
            sleep_time = self.window - (now - self.requests[0])
            print(f"Rate limit: sleeping {sleep_time:.2f}s")
            time.sleep(sleep_time)
            self.requests.popleft()
        
        self.requests.append(now)

# Demo: 5 requests per 10 seconds
limiter = RateLimiter(max_requests=5, window_seconds=10)

for i in range(7):
    limiter.acquire()
    print(f"Request {i+1} at {datetime.now().strftime('%H:%M:%S')}")
    
print("✓ Rate limiting working")

## Step 4: Batch Operations

In [ ]:
def batch_create_tokens(client, count: int, batch_size: int = 10):
    """Create tokens in batches with progress."""
    created = []
    
    for i in range(0, count, batch_size):
        batch = []
        
        for j in range(min(batch_size, count - i)):
            response = client.post("/tokens", json={
                "position": [float(i+j)]*8,
                "radius": 1.0,
                "weight": 1.0
            })
            batch.append(response.json())
        
        created.extend(batch)
        print(f"✓ Created batch {i//batch_size + 1}: {len(batch)} tokens")
    
    return created

# Create 25 tokens in batches of 10
tokens = batch_create_tokens(client, count=25, batch_size=10)
print(f"\n✓ Total created: {len(tokens)} tokens")

## Step 5: Error Handling Patterns

In [ ]:
from enum import Enum

class ErrorType(Enum):
    NETWORK = "network"
    AUTH = "auth"
    VALIDATION = "validation"
    NOT_FOUND = "not_found"
    SERVER = "server"

def handle_api_error(error: requests.exceptions.RequestException) -> ErrorType:
    """Classify and handle API errors."""
    if isinstance(error, requests.exceptions.ConnectionError):
        print("❌ Network error: Server unreachable")
        return ErrorType.NETWORK
    
    if isinstance(error, requests.exceptions.HTTPError):
        status = error.response.status_code
        
        if status == 401:
            print("❌ Authentication failed")
            return ErrorType.AUTH
        elif status == 404:
            print("❌ Resource not found")
            return ErrorType.NOT_FOUND
        elif status == 422:
            print(f"❌ Validation error: {error.response.json()}")
            return ErrorType.VALIDATION
        elif status >= 500:
            print("❌ Server error")
            return ErrorType.SERVER
    
    print(f"❌ Unknown error: {error}")
    return ErrorType.NETWORK

# Demo error handling
try:
    client.get("/tokens/999999")  # Non-existent token
except requests.exceptions.HTTPError as e:
    error_type = handle_api_error(e)
    print(f"Classified as: {error_type.value}")

## Step 6: Cleanup

In [ ]:
# Delete all tokens
response = client.get("/tokens")
all_tokens = response.json()

for token in all_tokens:
    client.delete(f"/tokens/{token['token_id']}")

print(f"✓ Cleaned up {len(all_tokens)} tokens")

## Summary

✅ **API Client** - Production-ready with auto-refresh  
✅ **Token management** - Automatic refresh before expiry  
✅ **Retry logic** - Exponential backoff on failures  
✅ **Rate limiting** - Token bucket algorithm  
✅ **Batch operations** - Efficient bulk processing  
✅ **Error handling** - Classify and recover from errors  

## Key Takeaways

1. **Always use token refresh** before expiry (30s buffer)
2. **Implement retry logic** with exponential backoff
3. **Respect rate limits** to avoid 429 errors
4. **Batch operations** reduce network overhead
5. **Error classification** enables smart recovery

---

**Next:** Advanced Tutorial 1 - Performance Optimization